todo:
1. probar coherencia con distintos hiperparametros (o simplemente decidirse por uno -> me gusta esta opción)
2. generar los ministerios (clusterizacion de topicos - > usando hierarchical topics es más simple, sin embargo, algunos topícos (de niveles más altos) presentan ruido y fusionan temas totalmente distintos (e.g. futbol + salud???))
3. generar dataset noticias y dataset topics, empezar a contextualizar los datos en el tiempo y por comuna. 
4. estudiar sobre knowledge graph -> me tinca mucho :p


In [1]:
# prevent huge warning messages of bertmodel 
import warnings
warnings.filterwarnings("ignore") 

from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()

import pandas as pd

import sys
sys.path.append('scripts/')

About [preprocessing]( https://github.com/MaartenGr/BERTopic/issues/40), in words of Maarten Grootendorst, author of BERTopic:


_"In general, no, you do not need to preprocess your data. Like you said, keeping the original structure of the text is especially important for transformer-based models to understand the context._

_However, there are exceptions to this. For example, if you were to have scraped documents with a lot of html tags, then it might be beneficial to remove those as they do not provide any interesting context."_

In [2]:
from preprocess import filter_by_media
from preprocess import cluster_by_month
from preprocess import find_cities

df = pd.read_csv("data/loslagos-comunas.csv")
df = cluster_by_month(filter_by_media(df))
df = df.drop_duplicates(subset='content', keep="first")
df.drop(columns=['comuna'], axis=1, inplace=True)
df['cities'] =  df.content.progress_apply(lambda x: find_cities(str(x)))
docs = df.content.tolist()

print("number of news:", len(df))

100%|██████████████████████████████████████████████████████████████████████████| 26127/26127 [00:19<00:00, 1318.22it/s]

number of news: 26127


In [14]:
df.media_outlet.value_counts()

laestrelladechiloe     6262
elaustral              4828
elllanquihue           4125
radiosago              2319
elcalbucano            1789
elheraldoaustral       1442
elinsular              1153
seminariolocal          804
elrepuertero            726
elvacanudo              702
elhuemul                386
radioacogida            352
radiopudeto             345
enlanoticia             317
fresiaahora             134
elquellonino            133
segundos33               95
radioestrelladelmar      93
soypuertomontt           62
soyosorno                56
soychiloe                 4
Name: media_outlet, dtype: int64

### "Document Clustering" with [BERTopic](https://github.com/MaartenGr/BERTopic) (+[SentenceTransformer](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) +[Word Embeddings](https://github.com/dccuchile/spanish-word-embeddings))

In [4]:
from gensim.models import KeyedVectors
from bertopic.backend import WordDocEmbedder
from sentence_transformers import SentenceTransformer

ft  = KeyedVectors.load_word2vec_format("data/SBW-vectors-300-min5.bin.gz", binary=True) 
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
word_doc_embedder = WordDocEmbedder(embedding_model=embedding_model, word_embedding_model=ft)

In [5]:
from topic_modeling import model_definition

topic_model = model_definition(word_doc_embedder) #word_doc_embedder
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'diversity': 0.2,
 'embedding_model': <bertopic.backend._word_doc.WordDocEmbedder at 0x1ec00d77788>,
 'hdbscan_model': HDBSCAN(min_cluster_size=10, min_samples=5, prediction_data=True),
 'language': None,
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'seed_topic_list': None,
 'top_n_words': 15,
 'umap_model': UMAP(metric='cosine', min_dist=0.0, n_components=5, random_state=42),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True}

In [6]:
topics, probs = topic_model.fit_transform(docs)

clusters = topic_model.get_topic_info()

Batches:   0%|          | 0/817 [00:00<?, ?it/s]

2022-12-15 00:21:59,789 - BERTopic - Transformed documents to Embeddings
2022-12-15 00:23:01,287 - BERTopic - Reduced dimensionality
2022-12-15 00:29:07,606 - BERTopic - Clustered reduced embeddings


Generamos un dataframe con los clusters obtenidos y extraemos sus tokens más significativos

In [7]:
clusters['most_freq_tokens'] = clusters.Topic.progress_apply(lambda x: topic_model.get_topic(x))

100%|███████████████████████████████████████████████████████████████████████████████| 478/478 [00:02<00:00, 166.38it/s]


In [8]:
clusters

,Topic,Count,Name,most_freq_tokens
0,-1,10748,-1_parte_personas_ser_además,"[(parte, 0.001092095937316581), (personas, 0.0..."
1,0,486,0_ucrania_rusia_ruso_rusa,"[(ucrania, 0.016083293518496796), (rusia, 0.01..."
2,1,385,1_accidente_conductor_vehículo_colisión,"[(accidente, 0.013586127284869442), (conductor..."
3,2,217,2_m谩s_chilo茅_tambi茅n_regi贸n,"[(m谩s, 0.006314027022407641), (chilo茅, 0.00473..."
4,3,201,3_comisión_constitucional_pleno_constitución,"[(comisión, 0.007116913316260138), (constituci..."
...,...,...,...,...
473,472,10,472_indígenas_familias indígenas_indígena_fami...,"[(indígenas, 0.020440527795508104), (familias ..."
474,473,10,473_partido_minutos_gol_puesto 10 campeonato,"[(partido, 0.009774374544295871), (minutos, 0...."
475,474,10,474_comisión_autonomía_autónomas_territorial,"[(comisión, 0.011075912597803187), (autonomía,..."
476,475,10,475_departamento gobierno_departamento gobiern...,"[(departamento gobierno, 0.017480715414542235)..."


In [9]:
clusters.to_csv('data/clusters.csv', index=False)

Etiquetamos las noticias con sus clusters

In [11]:
df['topic'] = ""

# label each row with his topic
labels=[]
for item in topic_model.generate_topic_labels():
    item.partition("_")[2]
    labels.append(item)

count = 0
for doc in tqdm(docs):  
    df.at[df.index[df['content'] == doc], 'topic'] = labels[topics[count]+1]
    count+=1

100%|███████████████████████████████████████████████████████████████████████████| 26127/26127 [01:21<00:00, 322.26it/s]


In [12]:
df.to_csv('data/labeled_news.csv', index=False)

In [13]:
df.head(4)

,date,media_outlet,url,title,text,content,date_clustering,cities,topic
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...,reconocen guardaparques región lagos actores c...,2021-10,"[puyehue, chaiten]",508_guardaparques_parque nacional_parque
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...,nuevos materiales comienza plan piloto saltos ...,2021-10,[puerto varas],489_good wood_wood_material reciclado
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...,centro salud familiar cesfam puerto varas invi...,2021-10,[puerto varas],84_cáncer_mama_cáncer mama
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...,alcalde tomás gárate presidió primera vez octa...,2021-10,"[castro, puerto varas]",328_gobierno terreno_bienes nacionales_trámites
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",galería arte machacoya realizará remate obras ...,2021-10,[],109_artista_arte_bosque nativo


#### Evaluation: Coherence Score

There is no one way to determine whether the coherence score is good or bad. The score and its value depends on the data that it's calculated from. For instance, in one case, the score of 0.5 might be good enough but in another case not acceptable. The only rule is that we want to **maximize** the score.

Usually, the coherence score will increase with the number of topics . This increase will become smaller as the number of topics get higher. The trade-off between the number of topics and coherence score can be achieved using the so-called elbow technique. The method implies plotting coherence score as a function of number of topics. We use the elbow of the curve to select the number of topics.

The idea behind this method is that we want to choose a point after which the diminishing increase of coherence score is no longer worth the additional increase of number of topics.

In [15]:
#from coherence_eval import cv_umass_npmi

#cv_coherence, umass_coherence, c_npmi_coherence = cv_umass_npmi(docs, topics, topic_model)

In [ ]:
#cv_coherence, umass_coherence, c_npmi_coherence

#### Hierarchical clustering

In [ ]:
from scipy.cluster import hierarchy as sch

# Hierarchical topics
linkage_function = lambda x: sch.linkage(x, 'ward', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(docs, linkage_function=linkage_function)

In [ ]:
pd.set_option("display.max_columns", 20, 'display.max_colwidth', 50)
hierarchical_topics.head(4)

In [ ]:
fig=topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
fig.write_image("img/htopics2.png")
fig

<img src="https://raw.githubusercontent.com/rickiwasho/proyecto-titulo/main/img/htopics2xddddd.png">

In [ ]:
#topic_model.save("out/save2", save_embedding_model=True)

In [ ]:
#my_model = BERTopic.load("out/save2")
#new_topics, new_probs = my_model.transform(docs)
#my_model.get_topic_info()

#### Topics over time

In [ ]:
timestamps = df.date.tolist()

# Es muy costoso (creo que se debe al word embedding)
topics_over_time = topic_model.topics_over_time(docs=docs, 
                                                timestamps=timestamps, 
                                                global_tuning=False, 
                                                evolution_tuning=False, 
                                                nr_bins=20)

In [ ]:
topics_over_time.head(4)

In [ ]:
fig = topic_model.visualize_topics_over_time(topics_over_time, topics=art)
fig.write_image("img/dtmt2v.png")

<img src="https://raw.githubusercontent.com/rickiwasho/proyecto-titulo/main/img/dtmt2v.png">

### 5 _most important_ keywords of documents using [KeyBERT](https://github.com/MaartenGr/KeyBERT) (+[Word Embeddings](https://github.com/dccuchile/spanish-word-embeddings))

In [ ]:
from keyword_extraction import extract_ngram_keywords

df['2gram_keywords'] = extract_ngram_keywords((2,2), ft, docs)
df['3gram_keywords'] = extract_ngram_keywords((3,3), ft, docs)

### Sentiment Analysis using [BETO](https://huggingface.co/finiteautomata/beto-sentiment-analysis?text=Te+quiero.+Te+amo.) + Sentiment Analysis/Emotion Analysis using [roBERTuito](https://huggingface.co/pysentimiento/robertuito-sentiment-analysis?text=Te+quiero.+Te+amo.)

In [ ]:
# BETO
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model_name = "finiteautomata/beto-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
# roBERTuito
from pysentimiento import create_analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
emotion_analyzer = create_analyzer(task="emotion", lang="es")

In [ ]:
from sentiment_analysis import sentiment_emotion_analysis
df = sentiment_emotion_analysis(df, sentiment_analyzer, emotion_analyzer, nlp)

In [ ]:
df.head(5)